<a href="https://colab.research.google.com/github/yudhiesh/PyTorch/blob/master/myPersonalityTextRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers==3
!pip install pytorch-lightning

In [41]:
import torch 
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix, classification_report
from torch import cuda

In [42]:
PATH = "/content/drive/MyDrive/BLI_Data/data.csv"
df = pd.read_csv(PATH)

In [43]:
df.head(10)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
5,b7b7764cfa1c523e4e93ab2a79a946c4,www.thejokerblogs.com,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/16/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
6,b7b7764cfa1c523e4e93ab2a79a946c4,"saw a nun zombie, and liked it. Also, *PROPNAM...",2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/27/09 05:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
7,b7b7764cfa1c523e4e93ab2a79a946c4,is in Kentucky. 421 miles into her 1100 mile j...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/18/09 06:34 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
8,b7b7764cfa1c523e4e93ab2a79a946c4,was about to finish a digital painting before ...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/09/09 02:58 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
9,b7b7764cfa1c523e4e93ab2a79a946c4,is celebrating her new haircut by listening to...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/07/09 11:41 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


In [44]:
cols = ["STATUS", "sEXT","sNEU","sAGR","sCON","sOPN"]
cols_normalize = ["sEXT","sNEU","sAGR","sCON","sOPN"]

In [45]:
df = df[cols]

In [46]:
df.head(10)

,STATUS,sEXT,sNEU,sAGR,sCON,sOPN
0,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4
1,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4
2,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4
3,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4
4,is home. <3,2.65,3.0,3.15,3.25,4.4
5,www.thejokerblogs.com,2.65,3.0,3.15,3.25,4.4
6,"saw a nun zombie, and liked it. Also, *PROPNAM...",2.65,3.0,3.15,3.25,4.4
7,is in Kentucky. 421 miles into her 1100 mile j...,2.65,3.0,3.15,3.25,4.4
8,was about to finish a digital painting before ...,2.65,3.0,3.15,3.25,4.4
9,is celebrating her new haircut by listening to...,2.65,3.0,3.15,3.25,4.4


In [47]:
# Normalize the values of all the scores
df[cols_normalize] = df[cols_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [48]:
df.sample(10)

,STATUS,sEXT,sNEU,sAGR,sCON,sOPN
7289,"""We Americans think you have to accomplish eve...",0.817439,0.262857,0.453731,0.766197,0.938182
6300,I now have a master's degree - ABT (all but th...,0.386921,0.642857,0.402985,0.507042,0.818182
5398,is working on new profile pic as E-28.,0.659401,0.314286,0.611940,0.718310,0.709091
5798,done with my paper finally =]],0.536785,0.442857,0.507463,0.788732,0.454545
2930,so sick kill me now,0.659401,0.228571,0.567164,0.704225,0.563636
2112,Off to see whats left of the Beach Boys (I go ...,0.817439,0.214286,0.752239,0.625352,0.781818
2129,Nothin's equivalent to a New York state of mind.,0.817439,0.214286,0.752239,0.625352,0.781818
4363,was not impressed with Gossip Girl tonight. It...,0.850136,0.357143,0.417910,0.619718,0.600000
1429,never going home during the weekday ever again...,0.495913,0.671429,0.522388,0.211268,0.727273
1423,Went apple picking during hte weekend with fam...,0.495913,0.671429,0.522388,0.211268,0.727273


In [49]:
df['lists'] = df[df.columns[1:6]].values.tolist()

In [50]:
df = df[['STATUS','lists']]

In [51]:
df.head(10)

,STATUS,lists
0,likes the sound of thunder.,"[0.35967302452316074, 0.5, 0.44776119402985076..."
1,is so sleepy it's not even funny that's she ca...,"[0.35967302452316074, 0.5, 0.44776119402985076..."
2,is sore and wants the knot of muscles at the b...,"[0.35967302452316074, 0.5, 0.44776119402985076..."
3,likes how the day sounds in this new song.,"[0.35967302452316074, 0.5, 0.44776119402985076..."
4,is home. <3,"[0.35967302452316074, 0.5, 0.44776119402985076..."
5,www.thejokerblogs.com,"[0.35967302452316074, 0.5, 0.44776119402985076..."
6,"saw a nun zombie, and liked it. Also, *PROPNAM...","[0.35967302452316074, 0.5, 0.44776119402985076..."
7,is in Kentucky. 421 miles into her 1100 mile j...,"[0.35967302452316074, 0.5, 0.44776119402985076..."
8,was about to finish a digital painting before ...,"[0.35967302452316074, 0.5, 0.44776119402985076..."
9,is celebrating her new haircut by listening to...,"[0.35967302452316074, 0.5, 0.44776119402985076..."


In [52]:
device = "cuda" if cuda.is_available() else "cpu"
print(device)

cuda


In [59]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 1e-5
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [60]:
import random

seed_val = 42 #so our results/process is reproducible by whoever wants to reproduce
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val) #include for when using a GPU

In [61]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.STATUS = dataframe.STATUS
        self.targets = self.data.lists
        self.max_len = max_len

    def __len__(self):
        return len(self.STATUS)

    def __getitem__(self, index):
        STATUS = str(self.STATUS[index])
        STATUS = " ".join(STATUS.split())

        inputs = self.tokenizer.encode_plus(
            STATUS,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [62]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=seed_val)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (9917, 2)
TRAIN Dataset: (7934, 2)
TEST Dataset: (1983, 2)


In [63]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 1
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [64]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 5)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [66]:
def loss_fn(outputs, targets):
    #  mean absolute error (MAE)
    return torch.nn.L1Loss()(outputs, targets)

In [67]:
def train_fn(data_loader, model, optimizer, device, scheduler, epoch):
  # Put the model in training mode
  model.train()

  # loop over all the batches
  for i, d in enumerate(data_loader):
    ids = d["ids"]
    token_type_ids = d["token_type_ids"]
    mask = d["mask"]
    targets = d["targets"]

    # convert to device type
    ids = ids.to(device, dtype=torch.long)
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    mask = mask.to(device, dtype=torch.long)
    targets = targets.to(device, dtype=torch.float)

    # zero-grad the optimizer
    optimizer.zero_grad()
    outputs = model(
        ids=ids,
        mask=mask,
        token_type_ids=token_type_ids
    )
    # calculate the loss
    loss = loss_fn(outputs, targets)
    if i % 5000==0:
      print(f'Epoch: {epoch}, Loss:  {loss.item()}')
    # backward step the loss
    loss.backward()
    # step the optimizer
    optimizer.step()
    # step scheduler
    scheduler.step()

In [68]:
def eval_fn(data_loader, model, device):
  # put the model in eval mode
  model.eval()
  # initialize empty lists for targets and outputs
  fin_targets = []
  fin_outputs = []
  
  # no_grad to prevent unnecessary use of GPU memory
  with torch.no_grad():
    for d in data_loader:
      ids = d["ids"]
      token_type_ids = d["token_type_ids"]
      mask = d["mask"]
      targets = d["targets"]
  
      # convert to device type
      ids = ids.to(device, dtype=torch.long)
      token_type_ids = token_type_ids.to(device, dtype=torch.long)
      mask = mask.to(device, dtype=torch.long)
      targets = targets.to(device, dtype=torch.float)
  
      outputs = model(
          ids=ids,
          mask=mask,
          token_type_ids=token_type_ids
      )
  
      # convert targets to cpu and extend the final list
      targets = targets.cpu().detach()
      fin_targets.extend(targets.numpy().tolist())
      outputs = outputs.cpu().detach()
      fin_outputs.extend(outputs.numpy().tolist())
  return fin_outputs, fin_targets


In [69]:
optimizer = AdamW(
    model.parameters(),
    lr=LEARNING_RATE, #2e-5 > 5e-5: A HYPERPARAMETER
    eps=1e-8
)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(training_loader) * EPOCHS
)

In [70]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

for epoch in range(EPOCHS):
  train_fn(training_loader,model, optimizer, device, scheduler, epoch)
  outputs, targets = eval_fn(testing_loader, model, device)
  print('Mean squared error: %.2f'
        % mean_squared_error(targets, outputs))
  print('Mean absolute error: %.2f'
        % mean_absolute_error(targets, outputs))
  print('Coefficient of determination: %.2f'
        % r2_score(targets, outputs))

Epoch: 0, Loss:  0.46311861276626587
Mean squared error: 0.05
Mean absolute error: 0.17
Coefficient of determination: 0.01
Epoch: 1, Loss:  0.17312714457511902
Mean squared error: 0.05
Mean absolute error: 0.17
Coefficient of determination: -0.00
Epoch: 2, Loss:  0.15762853622436523
Mean squared error: 0.05
Mean absolute error: 0.17
Coefficient of determination: 0.01
Epoch: 3, Loss:  0.1380726546049118
Mean squared error: 0.04
Mean absolute error: 0.17
Coefficient of determination: 0.02
Epoch: 4, Loss:  0.16709309816360474
Mean squared error: 0.04
Mean absolute error: 0.16
Coefficient of determination: 0.03
Epoch: 5, Loss:  0.11789919435977936
Mean squared error: 0.05
Mean absolute error: 0.17
Coefficient of determination: 0.00
Epoch: 6, Loss:  0.14585135877132416
Mean squared error: 0.05
Mean absolute error: 0.17
Coefficient of determination: -0.01
Epoch: 7, Loss:  0.15716253221035004
Mean squared error: 0.05
Mean absolute error: 0.17
Coefficient of determination: -0.00
Epoch: 8, Loss